In [32]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time
from scipy.sparse import csc_matrix,diags,lil_matrix,vstack
from scipy.sparse.linalg import spsolve,inv
from scipy.linalg import det
from scipy.spatial import Delaunay
from IPython.display import display, HTML,IFrame
import scipy.sparse as sp

interactive = True
use_cholesky = False
filename = 'woody-hi'
html=False
triangulation_methods=2 # 0 for scipy, 1 for triangle (removing unnecessary triangular), 2 for tiangle with random sample points

In [33]:
if interactive:
    html=False
if html:
    mp.website()
file_counter = 0
def htmlit(plot,method=1):
    if method==1:
        global file_counter
        unique_filename = f"./htmls/plot_{filename}{file_counter}.html"
        file_counter += 1
        plot.save(unique_filename)
        return unique_filename
    else:
        html = plot.to_html(imports=True, html_frame=False)
        return HTML(html)

In [34]:
v, f = igl.read_triangle_mesh(f'data/{filename}.off')
cage_verts=np.load("data/woody-hi.cage.npy")
v -= v.min(axis=0)
v /= v.max()

In [35]:
if triangulation_methods==0:
    tri_verts=np.vstack((cage_verts,v))[:,:2]
    temp=Delaunay(tri_verts)
    tri_verts,tri_faces=temp.points,temp.simplices
    padding=np.zeros((tri_verts.shape[0],1))
    tri_verts=np.hstack((tri_verts,padding))
    p=mp.plot(tri_verts,tri_faces,shading={"wireframe":True})
elif triangulation_methods==1:
    import triangle
    seg=[]
    for i in range(cage_verts.shape[0]):
        seg.append([i,(i+1)%cage_verts.shape[0]])
    tri_verts=np.vstack((cage_verts,v))[:,:2]
    t = triangle.triangulate({'vertices': tri_verts,'segments':seg},'p')
    tri_verts=np.array(t['vertices'])
    tri_faces=np.array(t['triangles'])
    padding=np.zeros((tri_verts.shape[0],1))
    tri_verts=np.hstack((tri_verts,padding))
    p=mp.plot(tri_verts,tri_faces,shading={"wireframe":True})
elif triangulation_methods==2:    
    import triangle
    seg=[]
    for i in range(cage_verts.shape[0]):
        seg.append([i,(i+1)%cage_verts.shape[0]])
    t = triangle.triangulate({'vertices': cage_verts[:,:2],'segments':seg},'pa0.01')
    tri_verts=np.array(t['vertices'])
    tri_faces=np.array(t['triangles'])
    padding=np.zeros((tri_verts.shape[0],1))
    tri_verts=np.hstack((tri_verts,padding))
    print(tri_verts.shape)
    p=mp.plot(tri_verts,tri_faces,shading={"wireframe":True})

(59, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4299999…

In [36]:
handle_vertex_positions = cage_verts.copy()
pos_f_saver = np.zeros((cage_verts.shape[0], 3))
def pos_f(s,x,y,z):
    #slices = (labels==s)
    #r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = cage_verts[s] + np.array([x,y,z])
    handle_vertex_positions[s] = v_slice
    pos_f_saver[s - 1] = [x,y,z]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    #v_deformed = add_detail(v_deformed)
    print(v_deformed.shape)
    global lines_obj
    p.update_object(vertices = v_deformed,colors=np.array(weights[:,s]))
    p.remove_object(lines_obj)

    lines_obj=p.add_lines(handle_vertex_positions,np.vstack((handle_vertex_positions[1:],handle_vertex_positions[0])))
    #p.add_lines(handle_vertex_positions)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
#pos_f.deformer = lambda x:x

In [37]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(cage_verts.shape[0]))
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    #rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    #widgets_dict.update(rotate_widget)
    return widgets_dict

In [47]:
def mybarycentric(p, a, b, c):
    v0 = b - a
    v1 = c - a
    v2 = p - a
    d00 = np.dot(v0, v0)
    d01 = np.dot(v0, v1)
    d11 = np.dot(v1, v1)
    d20 = np.dot(v2, v0)
    d21 = np.dot(v2, v1)
    denom = d00 * d11 - d01 * d01
    v = (d11 * d20 - d01 * d21) / denom
    w = (d00 * d21 - d01 * d20) / denom
    u = 1.0 - v - w
    return np.array([u,v,w])

def find_barycentric_triangle(obj_verts,cage_verts,cage_faces):
    index=np.zeros(obj_verts.shape[0],dtype="int")
    weight=np.zeros((obj_verts.shape[0],3))
    for i in range(obj_verts.shape[0]):
        for j in range(cage_faces.shape[0]):
            res=mybarycentric(obj_verts[i][:2],cage_verts[cage_faces[j,0]][:2],cage_verts[cage_faces[j,1]][:2],cage_verts[cage_faces[j,2]][:2])
            if res[0]>=0 and res[1]>=0 and res[2]>=0:
                index[i]=j
                weight[i]=res
                break
    return index,weight
    
def cagesolver_2d(tri_verts,tri_faces,cage_verts,obj_verts):
    
    weights_res=np.zeros((obj_verts.shape[0],cage_verts.shape[0]))
    mass_mat=igl.massmatrix(tri_verts,tri_faces)
    M=mass_mat
    cot_mat=igl.cotmatrix(tri_verts,tri_faces)
    L=inv(M)@cot_mat
    Lff=L[cage_verts.shape[0]:,:][:,cage_verts.shape[0]:]
    Lfc=L[cage_verts.shape[0]:,:][:,:cage_verts.shape[0]]
    xc=np.ones(cage_verts.shape[0])
    xc=diags(xc)
    result=spsolve(Lff,-Lfc@xc) # result weight is for any cage points not on boundary
    if triangulation_methods<=1:
        return result.toarray()
    print(np.eye(cage_verts.shape[0]).shape)
    print(result.shape)
    result=np.vstack((np.eye(cage_verts.shape[0]),result.toarray()))
    index,barycentric_weight=find_barycentric_triangle(obj_verts,tri_verts,tri_faces)
    for i in range(obj_verts.shape[0]):
        i_in_face=index[i]
        face_v0=tri_faces[i_in_face,0]
        face_v1=tri_faces[i_in_face,1]
        face_v2=tri_faces[i_in_face,2]
        weights_res[i]=(result[face_v0]*barycentric_weight[i,0]+result[face_v1]*barycentric_weight[i,1]+result[face_v2]*barycentric_weight[i,2])
    print(index[0])
    print(barycentric_weight[0])
    
    return weights_res
    
def cage_deformer(tri_verts):
    res=weights@tri_verts
    return res
pos_f.deformer=cage_deformer

In [48]:
weights=cagesolver_2d(tri_verts,tri_faces,cage_verts,v)

(21, 21)
(38, 21)
77
[0.43086378 0.31011267 0.25902356]


In [30]:
weights.shape
v.shape

(2642, 3)

In [58]:
## Widget UI
if interactive==True:
    p = mp.plot(v, f, c=weights[:,0])
    p.add_points(tri_verts[tri_faces[0]],shading={"point_size": 0.1})
    print(v[0])
    p.add_points(v[0][None,:],shading={"point_size": 0.1})
    lines_obj=p.add_lines(handle_vertex_positions,np.vstack((handle_vertex_positions[1:],handle_vertex_positions[0])))

    iw.interact(pos_f, **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4306930…

[0.00123762 0.61138614 0.        ]


interactive(children=(Dropdown(description='s', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,…